In [ ]:
import numpy as np
from typing import Tuple, Callable, List

from qiskit import QuantumCircuit
from qiskit.circuit.library import UnitaryGate
from qiskit.quantum_info import Operator
from qiskit_aer.primitives import Sampler


In [ ]:
class QSVT2x2:
    """
    2x2 Hermitian A에 대한 QSVT 실험용 클래스.

    - A: 2x2 Hermitian
    - 내부에서 A에 대한 (α,1)-block-encoding U(A)를 만들고,
      ancilla + system 2큐빗 회로로 들고 있음.
    """

    def __init__(self, A: np.ndarray):
        self.A = np.asarray(A, dtype=complex)
        self.block_circuit, self.alpha = self._block_encoding_hermitian_2x2(self.A)
        self.num_qubits = 2          # ancilla + system
        self.backend = Sampler()     # 기본 AER sampler

    @staticmethod
    def _block_encoding_hermitian_2x2(A: np.ndarray) -> Tuple[QuantumCircuit, float]:
        """
        2x2 Hermitian 행렬 A 에 대한 (α,1) block-encoding 유니터리 U 생성.

        q[0] : ancilla
        q[1] : system

        (⟨0|⊗I) U (|0⟩⊗I) = A / α  를 *정확히* 만족하게 만든다.
        """
        A = np.asarray(A, dtype=complex)

        # 1) 체크: 2x2, Hermitian
        if A.shape != (2, 2):
            raise ValueError("A는 2x2 행렬이어야 함.")
        if not np.allclose(A, A.conj().T):
            raise ValueError("A가 Hermitian이 아님 (A != A^†).")

        # 2) α = 최대 singular value = 최대 |eigenvalue| (Hermitian 이므로 동일)
        w, _ = np.linalg.eigh(A)
        alpha = float(np.max(np.abs(w)))
        if alpha == 0.0:
            raise ValueError("A가 영행렬이라 block-encoding 의미가 없음.")

        # 스케일된 연산자 \tilde{A} = A / α
        At = A / alpha

        # 3) 보조 함수: 2x2 양의 반정부호 행렬의 sqrt 계산
        def sqrt_psd(M: np.ndarray) -> np.ndarray:
            # Hermitian 보정
            M = 0.5 * (M + M.conj().T)
            ew, ev = np.linalg.eigh(M)
            # 수치 오차로 음수 아주 작게 나올 수 있으니 0으로 클립
            ew_clipped = np.clip(ew, 0.0, None)
            sqrt_ew = np.sqrt(ew_clipped)
            return ev @ np.diag(sqrt_ew) @ ev.conj().T

        I2 = np.eye(2, dtype=complex)

        # 4) 블록 구성에 필요한 루트들
        #   A가 Hermitian이면 At At^† = At^2 이고, 두 개가 같지만
        #   일반 공식을 그대로 쓰면 더 안전함.
        AA  = At @ At.conj().T
        AdA = At.conj().T @ At

        upper_right = sqrt_psd(I2 - AA)
        lower_left  = sqrt_psd(I2 - AdA)

        # 5) 블록 유니터리 U 구성
        # U = [[At, upper_right],
        #      [lower_left, -At^†]]
        U = np.zeros((4, 4), dtype=complex)
        # top-left
        U[0:2, 0:2] = At
        # top-right
        U[0:2, 2:4] = upper_right
        # bottom-left
        U[2:4, 0:2] = lower_left
        # bottom-right
        U[2:4, 2:4] = -At.conj().T

        # 6) unitary 체크
        if not np.allclose(U.conj().T @ U, np.eye(4), atol=1e-8):
            raise RuntimeError("구성한 U가 unitary가 아님. 구현 오류 가능.")

        from qiskit.circuit.library import UnitaryGate
        from qiskit import QuantumCircuit

        gate = UnitaryGate(U, label="U_block(A)")
        qc = QuantumCircuit(2, name="block_encoding")
        qc.append(gate, [0, 1])
        return qc, alpha

    def run(self, phis: List[float], shots: int = 1024):
        """
        주어진 phase 리스트로 QSVT-like 회로 실행하고 측정 결과 리턴.
        """
        qc = self.build_qsvt_circuit(phis)
        job = self.backend.run([qc], shots=shots)
        result = job.result()
        quasi = result.quasi_dists[0]
        counts = {bit: int(p * shots) for bit, p in quasi.items()}
        return counts

    def classical_apply(self, f: Callable[[float], complex]) -> np.ndarray:
        """
        f(λ)를 A의 고유값에 직접 적용해서 f(A)를 numpy로 계산.
        QSVT 결과 검증용.
        """
        w, V = np.linalg.eigh(self.A)
        f_eval = np.diag([f(lam) for lam in w])
        return V @ f_eval @ V.conj().T
    
    def build_qsp_circuit_precise(self, phis: list[float]) -> QuantumCircuit:
        """
        좀 더 QSP/QSVT 이론에 맞춘 '정석' 구조의 QSP 시퀀스.

        구조:
            - signal unitary S = (Z ⊗ I) · U_block
            - U_QSP(Φ) ≈ e^{i φ_0 Z} S e^{i φ_1 Z} S ... e^{i φ_d Z}

        여기서는:
            e^{i φ Z} 를 Rz(-2φ) 로 구현 (글로벌 페이즈 무시).
        """
        if len(phis) < 1:
            raise ValueError("최소 한 개 이상의 phase가 필요합니다.")

        qc = QuantumCircuit(self.num_qubits)

        # block encoding 유니터리 U
        U_op = Operator(self.block_circuit)

        # signal unitary S = (Z ⊗ I) · U
        # Z⊗I는 ancilla에 Z 하나 거는 거랑 같음
        # 회로 레벨에서는: Z(ancilla) -> U_block 순서로 하면 됨.
        def apply_signal_unitary(circ: QuantumCircuit):
            circ.z(0)                # (2Π - I) = Z on ancilla
            circ.append(U_op, [0, 1])  # U_block(A)

        # 1. 첫 phase: e^{i φ_0 Z} ~ Rz(-2 φ_0)
        qc.rz(-2 * phis[0], 0)

        # 2. 중간 단계: [S, e^{i φ_k Z}] 반복
        for k in range(1, len(phis)):
            apply_signal_unitary(qc)
            qc.rz(-2 * phis[k], 0)

        # 여기서는 ancilla + system 모두 측정 (테스트용)
        qc.measure_all()
        return qc

    def run_qsp_precise(self, phis: list[float], shots: int = 1024):
        """
        정석 QSP 구조로 만든 회로를 실행하고 측정 count 반환.
        """
        qc = self.build_qsp_circuit_precise(phis)
        job = self.backend.run([qc], shots=shots)
        result = job.result()
        quasi = result.quasi_dists[0]
        counts = {bit: int(p * shots) for bit, p in quasi.items()}
        return counts


if __name__ == "__main__":
    A = np.array([[0.3, 0.1],
                  [0.1, -0.5]], dtype=complex)

    qsvt = QSVT2x2(A)
    print("alpha =", qsvt.alpha)

    # 테스트용 phase 리스트 (랜덤/임의 값)
    phis = [0.1, -0.3, 0.2, 0.0]

    counts = qsvt.run_qsp_precise(phis, shots=2048)
    print("QSP precise counts:", counts)



C:\Users\sungw\AppData\Local\Temp\ipykernel_30604\1985155886.py:136: DeprecationWarning: Sampler has been deprecated as of Aer 0.15, please use SamplerV2 instead.
  qsvt = QSVT2x2(A)


alpha = 0.5123105625617661

A / alpha:
[[ 0.5855823 +0.j  0.1951941 +0.j]
 [ 0.1951941 +0.j -0.97597051+0.j]]

Top-left block of U:
[[ 0.5855823 +0.j  0.1951941 +0.j]
 [ 0.1951941 +0.j -0.97597051+0.j]]

차이 (A/alpha - top-left):
[[0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j]]

Measurement counts:
{0: 1013, 1: 9, 3: 2}

f(A) (classical, f(λ)=λ^2):
[[ 0.1 +0.j -0.02+0.j]
 [-0.02+0.j  0.26+0.j]]
